## KnowEnGSpreadsheetOrientedObject:   KSOO v00
    * Introspective Spreadsheet in the context of KnowEnG Pipeline Anaylsis
    * HBox Directory - File Selector
    


In [ ]:
# %cp ../../Samples_Clustering_Pipeline/test/run_dir/*.yml ../test/run_dir/

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import os
import sys
import time
from datetime import datetime

import numpy as np
import pandas as pd
import yaml
import knpackage.toolbox as kn

if os.path.isdir('/pipelines/InPheRNo/src'):
    src_dir = '/pipelines/InPheRNo/src'
else:
    src_dir = '../../InPheRNo/src'
    
sys.path.insert(1, src_dir)

from NoteBookYamlUI import ParameterSetWidgets
run_dir = '../test/run_dir/'

yaml_files_dir = os.path.abspath(run_dir)

psw = ParameterSetWidgets(yaml_files_dir)
psw.show_controls()